# Test du modèle de classification des prunes africaines

Ce notebook utilise les fonctions existantes dans le dépôt pour tester le modèle de classification des prunes africaines.

## 1. Configuration de l'environnement pour Google Colab

Commençons par cloner le dépôt GitHub et configurer l'environnement.

In [ ]:
# Vérifier si nous sommes dans Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print(f"Exécution dans Google Colab: {IN_COLAB}")

if IN_COLAB:
    # Cloner le dépôt GitHub
    !git clone https://github.com/CodeStorm-mbe/african-plums-classifier.git
    %cd african-plums-classifier
    
    # Installer les dépendances requises
    !pip install -r requirements.txt

In [ ]:
import os
import sys
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import glob
import random
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Ajouter le répertoire courant au chemin pour pouvoir importer nos modules
if IN_COLAB:
    # Dans Colab, nous sommes déjà dans le répertoire du projet
    if "/content/african-plums-classifier" not in sys.path:
        sys.path.append("/content/african-plums-classifier")
else:
    # En local, ajouter le répertoire parent
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

# Importer nos modules personnalisés
from data.data_preprocessing import preprocess_single_image
from models.model_architecture import get_model, TwoStageModel

# Définir les chemins
if IN_COLAB:
    # Dans Colab, créer les répertoires dans le dossier du projet cloné
    DATA_ROOT = "data/raw"
    MODELS_DIR = "models/saved"
    TEST_IMAGES_DIR = "data/test_images"
else:
    # En local
    DATA_ROOT = "../data/raw"
    MODELS_DIR = "../models/saved"
    TEST_IMAGES_DIR = "../data/test_images"

PLUM_DATA_DIR = os.path.join(DATA_ROOT, "plums")  # Sous-dossier pour les prunes
NON_PLUM_DATA_DIR = os.path.join(DATA_ROOT, "non_plums")  # Sous-dossier pour les non-prunes

# Créer les répertoires s'ils n'existent pas
os.makedirs(PLUM_DATA_DIR, exist_ok=True)
os.makedirs(NON_PLUM_DATA_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(TEST_IMAGES_DIR, exist_ok=True)

# Définir les paramètres
IMG_SIZE = 224
RANDOM_SEED = 42

# Fixer les seeds pour la reproductibilité
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Déterminer le device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de: {device}")

## 2. Téléchargement des modèles entraînés (pour Google Colab)

Si vous êtes dans Google Colab et que vous avez déjà entraîné des modèles, vous pouvez les télécharger ici.

In [ ]:
if IN_COLAB:
    from google.colab import files
    import shutil
    
    # Créer une fonction pour télécharger les modèles
    def upload_models():
        print("Veuillez télécharger les fichiers de modèle suivants:")
        print("1. detection_best_acc.pth")
        print("2. classification_best_acc.pth")
        print("3. two_stage_model_info.json")
        
        # Télécharger les fichiers
        uploaded = files.upload()
        
        # Déplacer les fichiers vers le répertoire des modèles
        for filename in uploaded.keys():
            shutil.move(filename, os.path.join(MODELS_DIR, filename))
            print(f"Fichier {filename} déplacé vers {MODELS_DIR}")
    
    # Vérifier si les modèles existent déjà
    model_files_exist = (
        os.path.exists(os.path.join(MODELS_DIR, 'detection_best_acc.pth')) and
        os.path.exists(os.path.join(MODELS_DIR, 'classification_best_acc.pth')) and
        os.path.exists(os.path.join(MODELS_DIR, 'two_stage_model_info.json'))
    )
    
    if not model_files_exist:
        print("Les fichiers de modèle n'existent pas. Vous pouvez les télécharger maintenant.")
        # Décommentez la ligne suivante pour télécharger les modèles
        # upload_models()
    else:
        print("Les fichiers de modèle existent déjà.")

## 3. Chargement du modèle

Chargeons le modèle à deux étapes que nous avons entraîné précédemment.

In [ ]:
def load_two_stage_model():
    """Charge le modèle à deux étapes à partir des fichiers sauvegardés."""
    # Vérifier si les fichiers nécessaires existent
    model_info_path = os.path.join(MODELS_DIR, 'two_stage_model_info.json')
    detection_model_path = os.path.join(MODELS_DIR, 'detection_best_acc.pth')
    classification_model_path = os.path.join(MODELS_DIR, 'classification_best_acc.pth')
    
    if not os.path.exists(model_info_path):
        raise FileNotFoundError(f"Le fichier d'informations du modèle n'existe pas: {model_info_path}")
    if not os.path.exists(detection_model_path):
        raise FileNotFoundError(f"Le fichier du modèle de détection n'existe pas: {detection_model_path}")
    if not os.path.exists(classification_model_path):
        raise FileNotFoundError(f"Le fichier du modèle de classification n'existe pas: {classification_model_path}")
    
    # Charger les informations du modèle
    with open(model_info_path, 'r') as f:
        model_info = json.load(f)
    
    # Extraire les informations
    detection_classes = model_info['detection_classes']
    classification_classes = model_info['classification_classes']
    model_architecture_info = model_info['model_info']
    
    # Créer les modèles
    detection_model = get_model(
        model_name=model_architecture_info['detection_model']['base_model'].split('_')[0] if '_' in model_architecture_info['detection_model']['base_model'] else 'standard',
        num_classes=len(detection_classes),
        base_model=model_architecture_info['detection_model']['base_model'].split('_')[1] if '_' in model_architecture_info['detection_model']['base_model'] else model_architecture_info['detection_model']['base_model'],
        pretrained=False
    )
    
    classification_model = get_model(
        model_name=model_architecture_info['classification_model']['base_model'].split('_')[0] if '_' in model_architecture_info['classification_model']['base_model'] else 'standard',
        num_classes=len(classification_classes),
        base_model=model_architecture_info['classification_model']['base_model'].split('_')[1] if '_' in model_architecture_info['classification_model']['base_model'] else model_architecture_info['classification_model']['base_model'],
        pretrained=False
    )
    
    # Charger les poids
    detection_model.load_state_dict(torch.load(detection_model_path, map_location=device))
    classification_model.load_state_dict(torch.load(classification_model_path, map_location=device))
    
    # Créer le modèle à deux étapes
    two_stage_model = TwoStageModel(
        detection_model, 
        classification_model, 
        detection_threshold=model_architecture_info['detection_threshold'] if 'detection_threshold' in model_architecture_info else 0.7
    )
    
    return two_stage_model, detection_classes, classification_classes

# Charger le modèle
try:
    model, detection_classes, classification_classes = load_two_stage_model()
    model.detection_model.to(device)
    model.classification_model.to(device)
    
    print("Modèle à deux étapes chargé avec succès!")
    print(f"Classes de détection: {detection_classes}")
    print(f"Classes de classification: {classification_classes}")
except Exception as e:
    print(f"Erreur lors du chargement du modèle: {e}")
    print("\nSi vous n'avez pas encore entraîné le modèle, veuillez d'abord exécuter le notebook d'entraînement.")
    
    # Créer des classes fictives pour pouvoir continuer le notebook
    detection_classes = ['plum', 'non_plum']
    classification_classes = ['ripe', 'unripe', 'damaged', 'diseased', 'overripe', 'healthy']
    print("\nUtilisation de classes fictives pour la démonstration:")
    print(f"Classes de détection: {detection_classes}")
    print(f"Classes de classification: {classification_classes}")

## 4. Création d'images de test

Créons des images de test synthétiques pour tester notre modèle.

In [ ]:
def create_test_images(force_create=False):
    """Crée des images de test synthétiques."""
    # Vérifier si des images existent déjà
    existing_images = glob.glob(os.path.join(TEST_IMAGES_DIR, "*.jpg"))
    if existing_images and not force_create:
        print(f"Des images de test existent déjà ({len(existing_images)} images). Utilisez force_create=True pour les remplacer.")
        return
    
    # Créer des images de test
    print("Création d'images de test synthétiques...")
    
    # Définir les couleurs pour les différentes classes
    colors = {
        "ripe": (150, 0, 0),      # Rouge foncé
        "unripe": (0, 150, 0),    # Vert foncé
        "damaged": (150, 100, 0), # Marron
        "diseased": (100, 0, 100),# Violet
        "overripe": (100, 0, 0),  # Rouge très foncé
        "healthy": (150, 50, 50)  # Rouge-rose
    }
    
    # Créer des images pour chaque classe de prune
    for cls, base_color in colors.items():
        for i in range(3):  # 3 images par classe
            # Ajouter un peu de variation aléatoire à la couleur
            color_var = [max(0, min(255, c + random.randint(-20, 20))) for c in base_color]
            
            # Créer une image
            img = Image.new('RGB', (224, 224), (255, 255, 255))
            pixels = img.load()
            
            # Dessiner un cercle approximatif avec la couleur
            center_x, center_y = 112, 112
            radius = 100 + random.randint(-10, 10)
            
            for x in range(img.width):
                for y in range(img.height):
                    dist = ((x - center_x) ** 2 + (y - center_y) ** 2) ** 0.5
                    if dist <= radius:
                        # Ajouter du bruit à chaque pixel
                        pixel_color = [max(0, min(255, c + random.randint(-10, 10))) for c in color_var]
                        pixels[x, y] = tuple(pixel_color)
            
            # Sauvegarder l'image
            img_path = os.path.join(TEST_IMAGES_DIR, f"test_{cls}_{i+1}.jpg")
            img.save(img_path)
    
    # Créer des images non-prune
    for i in range(5):  # 5 images non-prune
        # Couleur aléatoire qui n'est pas proche des couleurs de prune
        color = (random.randint(0, 100), random.randint(150, 255), random.randint(150, 255))
        
        # Créer une image
        img = Image.new('RGB', (224, 224), (255, 255, 255))
        pixels = img.load()
        
        # Dessiner une forme aléatoire (carré ou triangle)
        shape = random.choice(['square', 'triangle'])
        
        if shape == 'square':
            # Dessiner un carré
            size = random.randint(100, 150)
            top_left = (random.randint(0, 224-size), random.randint(0, 224-size))
            
            for x in range(top_left[0], top_left[0] + size):
                for y in range(top_left[1], top_left[1] + size):
                    if 0 <= x < 224 and 0 <= y < 224:
                        # Ajouter du bruit à chaque pixel
                        pixel_color = [max(0, min(255, c + random.randint(-10, 10))) for c in color]
                        pixels[x, y] = tuple(pixel_color)
        else:
            # Dessiner un triangle
            p1 = (random.randint(50, 174), random.randint(50, 174))
            p2 = (p1[0] + random.randint(30, 50), p1[1] + random.randint(30, 50))
            p3 = (p1[0] - random.randint(0, 30), p2[1])
            
            # Remplir le triangle (algorithme simple)
            min_x = min(p1[0], p2[0], p3[0])
            max_x = max(p1[0], p2[0], p3[0])
            min_y = min(p1[1], p2[1], p3[1])
            max_y = max(p1[1], p2[1], p3[1])
            
            for x in range(min_x, max_x + 1):
                for y in range(min_y, max_y + 1):
                    if 0 <= x < 224 and 0 <= y < 224:
                        # Vérification simple si le point est dans le triangle
                        if (x >= p1[0] and y >= p1[1] and x <= p2[0] and y <= p2[1]):
                            # Ajouter du bruit à chaque pixel
                            pixel_color = [max(0, min(255, c + random.randint(-10, 10))) for c in color]
                            pixels[x, y] = tuple(pixel_color)
        
        # Sauvegarder l'image
        img_path = os.path.join(TEST_IMAGES_DIR, f"test_non_plum_{i+1}.jpg")
        img.save(img_path)
    
    print(f"Images de test créées avec succès dans {TEST_IMAGES_DIR}")
    print(f"- 18 images de prunes (3 par classe)")
    print(f"- 5 images non-prune")

# Créer des images de test
create_test_images(force_create=True)

## 5. Test sur des images individuelles

Testons notre modèle sur quelques images individuelles.

In [ ]:
def predict_image(image_path, model, device, detection_classes, classification_classes):
    """Prédit la classe d'une image en utilisant la fonction preprocess_single_image du module data_preprocessing."""
    # Vérifier si le fichier existe
    if not os.path.exists(image_path):
        return {
            'error': f"Le fichier {image_path} n'existe pas."
        }
    
    try:
        # Prétraiter l'image
        image_tensor = preprocess_single_image(image_path)
        
        # Prédire
        is_plum, predicted_class_idx, probabilities = model.predict(image_tensor, device)
        
        # Préparer les résultats
        result = {
            'image_path': image_path,
            'is_plum': is_plum,
            'detection_class': detection_classes[0] if is_plum else detection_classes[1],
            'detection_confidence': float(probabilities[0]) if is_plum else float(probabilities[1])
        }
        
        # Si c'est une prune, ajouter les informations de classification
        if is_plum:
            result['classification_class'] = classification_classes[predicted_class_idx]
            result['classification_probabilities'] = {
                cls: float(prob) for cls, prob in zip(classification_classes, probabilities)
            }
        
        return result
    except Exception as e:
        return {
            'error': f"Erreur lors de la prédiction: {str(e)}"
        }

def visualize_prediction(image_path, prediction_result):
    """Visualise l'image avec sa prédiction."""
    # Vérifier s'il y a une erreur
    if 'error' in prediction_result:
        print(f"Erreur: {prediction_result['error']}")
        return
    
    # Charger l'image
    img = Image.open(image_path)
    
    # Afficher l'image avec la prédiction
    plt.figure(figsize=(10, 6))
    plt.imshow(img)
    plt.axis('off')
    
    # Préparer le titre
    if prediction_result['is_plum']:
        title = f"Prédiction: {prediction_result['classification_class']} (Confiance: {prediction_result['detection_confidence']:.2f})"
    else:
        title = f"Prédiction: Non-prune (Confiance: {prediction_result['detection_confidence']:.2f})"
    
    plt.title(title, fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # Si c'est une prune, afficher les probabilités de classification
    if prediction_result['is_plum'] and 'classification_probabilities' in prediction_result:
        # Extraire les probabilités
        classes = list(prediction_result['classification_probabilities'].keys())
        probs = list(prediction_result['classification_probabilities'].values())
        
        # Trier par probabilité décroissante
        sorted_idx = np.argsort(probs)[::-1]
        classes = [classes[i] for i in sorted_idx]
        probs = [probs[i] for i in sorted_idx]
        
        # Afficher le graphique des probabilités
        plt.figure(figsize=(10, 4))
        plt.bar(classes, probs, color='skyblue')
        plt.xlabel('Classe')
        plt.ylabel('Probabilité')
        plt.title('Probabilités de classification')
        plt.xticks(rotation=45)
        plt.ylim(0, 1)
        plt.tight_layout()
        plt.show()

In [ ]:
def test_individual_images():
    """Teste le modèle sur quelques images individuelles."""
    # Vérifier si le modèle est chargé
    if 'model' not in globals():
        print("Le modèle n'est pas chargé. Veuillez d'abord exécuter la cellule de chargement du modèle.")
        return
    
    # Vérifier s'il y a des images de test
    test_images = glob.glob(os.path.join(TEST_IMAGES_DIR, "*.jpg"))
    if not test_images:
        print(f"Aucune image de test trouvée dans {TEST_IMAGES_DIR}. Veuillez d'abord créer des images de test.")
        return
    
    # Sélectionner quelques images aléatoires
    num_images = min(5, len(test_images))
    selected_images = random.sample(test_images, num_images)
    
    # Tester chaque image
    for img_path in selected_images:
        print(f"\nTest de l'image: {os.path.basename(img_path)}")
        
        # Prédire
        prediction = predict_image(img_path, model, device, detection_classes, classification_classes)
        
        # Afficher les résultats
        if 'error' in prediction:
            print(f"Erreur: {prediction['error']}")
        else:
            if prediction['is_plum']:
                print(f"Résultat: Prune de type {prediction['classification_class']}")
                print(f"Confiance de détection: {prediction['detection_confidence']:.4f}")
            else:
                print(f"Résultat: Non-prune")
                print(f"Confiance: {prediction['detection_confidence']:.4f}")
        
        # Visualiser
        visualize_prediction(img_path, prediction)

# Tester sur des images individuelles
if 'model' in globals():
    test_individual_images()
else:
    print("Le modèle n'est pas chargé. Veuillez d'abord exécuter la cellule de chargement du modèle.")

## 6. Test sur un lot d'images

Testons maintenant notre modèle sur un lot d'images pour évaluer ses performances globales.

In [ ]:
def test_batch_images():
    """Teste le modèle sur un lot d'images."""
    # Vérifier si le modèle est chargé
    if 'model' not in globals():
        print("Le modèle n'est pas chargé. Veuillez d'abord exécuter la cellule de chargement du modèle.")
        return
    
    # Vérifier s'il y a des images de test
    test_images = glob.glob(os.path.join(TEST_IMAGES_DIR, "*.jpg"))
    if not test_images:
        print(f"Aucune image de test trouvée dans {TEST_IMAGES_DIR}. Veuillez d'abord créer des images de test.")
        return
    
    # Prédire pour toutes les images
    results = []
    for img_path in test_images:
        prediction = predict_image(img_path, model, device, detection_classes, classification_classes)
        if 'error' not in prediction:
            # Extraire la classe réelle à partir du nom de fichier
            filename = os.path.basename(img_path)
            if '_' in filename:
                parts = filename.split('_')
                if len(parts) >= 2:
                    true_class = parts[1]
                    # Vérifier si c'est une classe valide
                    if true_class in classification_classes or true_class == 'non':
                        prediction['true_class'] = 'non_plum' if true_class == 'non' else true_class
                    else:
                        prediction['true_class'] = 'unknown'
                else:
                    prediction['true_class'] = 'unknown'
            else:
                prediction['true_class'] = 'unknown'
            
            results.append(prediction)
    
    # Afficher les résultats
    print(f"Résultats pour {len(results)} images:")
    
    # Compter les prédictions correctes
    correct_detection = 0
    correct_classification = 0
    total_with_true_class = 0
    
    for result in results:
        if result['true_class'] != 'unknown':
            total_with_true_class += 1
            
            # Vérifier si la détection est correcte
            is_true_plum = result['true_class'] != 'non_plum'
            if result['is_plum'] == is_true_plum:
                correct_detection += 1
                
                # Si c'est une prune, vérifier si la classification est correcte
                if is_true_plum and result['classification_class'] == result['true_class']:
                    correct_classification += 1
    
    # Calculer les métriques
    if total_with_true_class > 0:
        detection_accuracy = correct_detection / total_with_true_class
        classification_accuracy = correct_classification / sum(1 for r in results if r['true_class'] != 'unknown' and r['true_class'] != 'non_plum')
        
        print(f"Précision de détection: {detection_accuracy:.4f} ({correct_detection}/{total_with_true_class})")
        print(f"Précision de classification: {classification_accuracy:.4f} ({correct_classification}/{sum(1 for r in results if r['true_class'] != 'unknown' and r['true_class'] != 'non_plum')})")
    else:
        print("Aucune image avec une classe réelle connue.")
    
    return results

# Tester sur un lot d'images
if 'model' in globals():
    batch_results = test_batch_images()
else:
    print("Le modèle n'est pas chargé. Veuillez d'abord exécuter la cellule de chargement du modèle.")

## 7. Visualisation des résultats

Visualisons les résultats de notre modèle sur le lot d'images.

In [ ]:
def visualize_batch_results(results):
    """Visualise les résultats du test sur un lot d'images."""
    if not results:
        print("Aucun résultat à visualiser.")
        return
    
    # Filtrer les résultats avec une classe réelle connue
    results_with_true_class = [r for r in results if r['true_class'] != 'unknown']
    
    if not results_with_true_class:
        print("Aucun résultat avec une classe réelle connue.")
        return
    
    # Préparer les données pour la matrice de confusion de détection
    y_true_detection = [1 if r['true_class'] != 'non_plum' else 0 for r in results_with_true_class]
    y_pred_detection = [1 if r['is_plum'] else 0 for r in results_with_true_class]
    
    # Calculer la matrice de confusion pour la détection
    cm_detection = confusion_matrix(y_true_detection, y_pred_detection)
    
    # Visualiser la matrice de confusion pour la détection
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_detection, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Non-prune', 'Prune'], 
                yticklabels=['Non-prune', 'Prune'])
    plt.xlabel('Prédiction')
    plt.ylabel('Vérité')
    plt.title('Matrice de confusion pour la détection')
    plt.tight_layout()
    plt.show()
    
    # Préparer les données pour la matrice de confusion de classification
    # Filtrer les résultats qui sont des prunes (vraies et prédites)
    plum_results = [r for r in results_with_true_class 
                   if r['true_class'] != 'non_plum' and r['is_plum']]
    
    if plum_results:
        # Obtenir toutes les classes uniques
        all_classes = sorted(list(set([r['true_class'] for r in plum_results] + 
                                  [r['classification_class'] for r in plum_results])))
        
        # Créer des mappages pour les indices
        class_to_idx = {cls: i for i, cls in enumerate(all_classes)}
        
        # Préparer les données
        y_true_classification = [class_to_idx[r['true_class']] for r in plum_results]
        y_pred_classification = [class_to_idx[r['classification_class']] for r in plum_results]
        
        # Calculer la matrice de confusion pour la classification
        cm_classification = confusion_matrix(y_true_classification, y_pred_classification, 
                                           labels=range(len(all_classes)))
        
        # Visualiser la matrice de confusion pour la classification
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm_classification, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=all_classes, 
                    yticklabels=all_classes)
        plt.xlabel('Prédiction')
        plt.ylabel('Vérité')
        plt.title('Matrice de confusion pour la classification')
        plt.tight_layout()
        plt.show()
    else:
        print("Pas assez de données pour la matrice de confusion de classification.")

# Visualiser les résultats du lot
if 'batch_results' in globals() and batch_results:
    visualize_batch_results(batch_results)
else:
    print("Aucun résultat de lot à visualiser. Veuillez d'abord exécuter le test sur un lot d'images.")

## 8. Téléchargement d'une image pour test (pour Google Colab)

Si vous êtes dans Google Colab, vous pouvez télécharger une image pour la tester.

In [ ]:
if IN_COLAB:
    from google.colab import files
    import shutil
    
    def upload_test_image():
        print("Veuillez télécharger une image à tester:")
        uploaded = files.upload()
        
        # Déplacer l'image vers le répertoire de test
        for filename in uploaded.keys():
            dest_path = os.path.join(TEST_IMAGES_DIR, filename)
            shutil.move(filename, dest_path)
            print(f"Image {filename} déplacée vers {dest_path}")
            return dest_path
        
        return None
    
    # Décommentez la ligne suivante pour télécharger une image de test
    # uploaded_image_path = upload_test_image()

## 9. Test sur une image spécifique

Testons le modèle sur une image spécifique.

In [ ]:
def test_specific_image(image_path):
    """Teste le modèle sur une image spécifique."""
    # Vérifier si le modèle est chargé
    if 'model' not in globals():
        print("Le modèle n'est pas chargé. Veuillez d'abord exécuter la cellule de chargement du modèle.")
        return
    
    # Vérifier si le fichier existe
    if not os.path.exists(image_path):
        print(f"Le fichier {image_path} n'existe pas.")
        return
    
    # Prédire
    prediction = predict_image(image_path, model, device, detection_classes, classification_classes)
    
    # Afficher les résultats
    if 'error' in prediction:
        print(f"Erreur: {prediction['error']}")
    else:
        if prediction['is_plum']:
            print(f"Résultat: Prune de type {prediction['classification_class']}")
            print(f"Confiance de détection: {prediction['detection_confidence']:.4f}")
        else:
            print(f"Résultat: Non-prune")
            print(f"Confiance: {prediction['detection_confidence']:.4f}")
    
    # Visualiser
    visualize_prediction(image_path, prediction)
    
    return prediction

# Si une image a été téléchargée dans Colab, la tester
if IN_COLAB and 'uploaded_image_path' in locals() and uploaded_image_path:
    test_specific_image(uploaded_image_path)
else:
    # Sinon, tester une image aléatoire du répertoire de test
    test_images = glob.glob(os.path.join(TEST_IMAGES_DIR, "*.jpg"))
    if test_images:
        random_image = random.choice(test_images)
        print(f"Test d'une image aléatoire: {os.path.basename(random_image)}")
        test_specific_image(random_image)

## 10. Conclusion

Dans ce notebook, nous avons utilisé les fonctions existantes des modules `data_preprocessing` et `model_architecture` pour :
1. Charger le modèle à deux étapes entraîné
2. Créer des images de test
3. Tester le modèle sur des images individuelles
4. Évaluer les performances du modèle sur un lot d'images
5. Visualiser les résultats

Ces étapes nous ont permis de comprendre les forces et les faiblesses de notre modèle, et d'identifier les domaines où il pourrait être amélioré.